<a href="https://colab.research.google.com/github/snpushpi/6.804-Computational-Cognitive-Science/blob/master/Particle_Filter_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import random
import math
import scipy.stats
from collections import Counter 

state_set = {'A','B','C','D'}
mu_dict = {'A':.2,'B':.4,'C':.6,'D':.8}
sigma = 0.1
def data_generate(alpha,T):
    '''Goal is generate T observations with state changing probability alpha.
    Step 1: Set Z_0 as a random sample from state list.
    Step 2: Repeat the following for T trials -
          i) Z_i = Z_i-1
          ii)Sample x randomly from [0,1]
          iii) If x<alpha replace Z_i with random sample {A,B,C,D}/Z_i-1
          iv)Sample stimulus y_i form a normal distribution with std sigma and mu_zi
    '''
    observation_list = []
    Z = [None]*(T+1)
    Z[0] = random.choice(tuple(state_set))
    for i in range(1,T+1):
        Z[i]=Z[i-1]
        x = random.uniform(0,1)
        if x<alpha:
            new_set = state_set-{Z[i-1]}
            Z[i]= random.choice(tuple(new_set))
        observation_list.append(random.gauss(mu_dict[Z[i]],sigma))        
    return observation_list,Z[1:]

def weight_based_sampling(S): #[[state,weight]]
    states=[e[0] for e in S]
    weights = [e[1] for e in S]
    state =  np.random.choice(states,p=weights)
    weight = None
    for elt in S:
        if elt[0]==state:
            weight = elt[1]
    return state,weight

def most_probable(S):
    weight_dict = {'A':0,'B':0,'C':0,'D':0}
    for state,weight in S:
        weight_dict[state]+=weight
    return max(weight_dict, key=weight_dict.get)

def particle_filter(observation_list,r,particle_num,T):
    '''1. Initialize a list (call it S_0) with a [particle_state,weight] as element(call each element [s_0,w_0]). Initially all weights are equal
       2. Do thw follwing for trial 1 to trial T- 
          i) Create an empty list call S_i for storing new particles. Also initialize a normalization constant eta with 0
          for i=1 to i=particle_num do the following - 
              ii) at trial j, consider the particle list S_{j-1} and sample a particle from this particle from this distribution based on weight w_{i-1}
              iii) CAll the sampled particle from step ii x_i, now change state with probability alpha, stay in the same state with the rest.
              Call this new particle x_i'
              iv) w_i = P(ith observation|x_i'), eta+=w_i
              v) S_i.append([x_i',w_i])
          for i=1 to i=particle_num:
              w_i/=w_i/eta
    '''
    S = [[random.choice(tuple(state_set)),1/particle_num] for i in range(particle_num)] #S=[[state,1/particle_num]]
    prediction_list = []
    for t in range(1,T+1): #
        S_new = [] 
        S_p = []
        eta = 0 
        eta_p = []
        for i in range(particle_num):
            state,weight = weight_based_sampling(S) #a particle 
            x1 = random.uniform(0,1) 
            if x1<r: #change state - (1-e**(-r*k)) =>
                new_set = state_set-{state}
                state= random.choice(tuple(new_set))
            S_p.append([state,weight])
            new_weight = scipy.stats.norm(mu_dict[state],sigma).pdf(observation_list[t-1])
            eta+= new_weight
            S_new.append([state,new_weight])
        S_p = [[elt[0],elt[1]]for elt in S_p]
        prediction_list.append(most_probable(S_p))
        S_new = [[elt[0],elt[1]/eta] for elt in S_new]
        S = S_new #S_t - S-
    return prediction_list

def count_accuracy(alpha,T,particle_num,r):
    observation_list,actual_list = data_generate(alpha,T)
    prediction_list = particle_filter(observation_list,r,particle_num,T)
    count = 0
    for i in range(len(observation_list)):
        if prediction_list[i]==actual_list[i]:
            count+=1
    return count/T
print(count_accuracy(0.08,1000,150,0.1))


0.844
